In [1]:
from konlpy.tag import Okt
import json
import os
from pprint import pprint
import nltk
import numpy as np

In [2]:
# 파일 로드를 위한 함수
def read_data(filename):
    with open(filename, 'r', encoding='UTF8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        # txt 파일의 헤더(id document label)는 제외하기
        data = data[1:]
    return data

train_data = read_data('ratings_train.txt') #데이터불러오기
test_data = read_data('ratings_test.txt') #데이터불러오기

In [3]:
# 1) morphs : 형태소 추출
# 2) pos : 품사 부착(Part-of-speech tagging)
# 3) nouns : 명사 추출
okt = Okt()

In [4]:
# 테스트
print(okt.pos(u'이 밤 그날의 반딧불을 당신의 창 가까이 보낼게요'))

[('이', 'Noun'), ('밤', 'Noun'), ('그날', 'Noun'), ('의', 'Josa'), ('반딧불', 'Noun'), ('을', 'Josa'), ('당신', 'Noun'), ('의', 'Josa'), ('창', 'Noun'), ('가까이', 'Noun'), ('보낼게요', 'Verb')]


In [5]:
def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

In [6]:
if os.path.isfile('train_docs.json'): #파일이존재하면
    with open('train_docs.json', encoding='UTF8') as f:
        train_docs = json.load(f)
    with open('test_docs.json', encoding='UTF8') as f:
        test_docs = json.load(f)
else:
    train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
    test_docs = [(tokenize(row[1]), row[2]) for row in test_data]
    # JSON 파일로 저장
    with open('train_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(train_docs, make_file, ensure_ascii=False, indent="\t")
    with open('test_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(test_docs, make_file, ensure_ascii=False, indent="\t")

In [7]:
pprint(train_docs[0]) #tokenize된 학습데이터 첫번째 테스트출력

[['아/Exclamation',
  '더빙/Noun',
  '../Punctuation',
  '진짜/Noun',
  '짜증나다/Adjective',
  '목소리/Noun'],
 '0']


In [8]:
tokens = [t for d in train_docs for t in d[0]]
print(len(tokens)) #학습데이터의 총 토큰개수

2159921


In [9]:
print(type(tokens)) #tokens의 타입은 list. 토큰들이 들어가있다.

<class 'list'>


In [10]:
text = nltk.Text(tokens, name='NMSC')

# 전체 토큰의 개수
print(len(text.tokens))

# 중복을 제외한 토큰의 개수
print(len(set(text.tokens)))

# 출현 빈도가 높은 상위 토큰 10개
pprint(text.vocab().most_common(10))

2159921
49895
[('./Punctuation', 67778),
 ('영화/Noun', 50818),
 ('하다/Verb', 41209),
 ('이/Josa', 38540),
 ('보다/Verb', 38538),
 ('의/Josa', 30188),
 ('../Punctuation', 29055),
 ('가/Josa', 26627),
 ('에/Josa', 26468),
 ('을/Josa', 23118)]


In [11]:
# 출현빈도가 높은 상위 토큰을 몇개 가져올건지 정하여 정확도높이기
# 시간이 꽤 걸립니다! 시간을 절약하고 싶으면 most_common의 매개변수를 줄여보세요.
# most_common(100) 의 수를 높일 수록 정확도가 올라갑니다.
selected_words = [f[0] for f in text.vocab().most_common(1000)]

selected_words

['./Punctuation',
 '영화/Noun',
 '하다/Verb',
 '이/Josa',
 '보다/Verb',
 '의/Josa',
 '../Punctuation',
 '가/Josa',
 '에/Josa',
 '을/Josa',
 '.../Punctuation',
 '도/Josa',
 '은/Josa',
 '들/Suffix',
 ',/Punctuation',
 '는/Josa',
 '없다/Adjective',
 '를/Josa',
 '있다/Adjective',
 '좋다/Adjective',
 '너무/Adverb',
 '?/Punctuation',
 '이/Determiner',
 '재밌다/Adjective',
 '정말/Noun',
 '것/Noun',
 '되다/Verb',
 '!/Punctuation',
 '진짜/Noun',
 '같다/Adjective',
 '적/Suffix',
 '으로/Josa',
 '이/Noun',
 '점/Noun',
 '아니다/Adjective',
 '않다/Verb',
 '로/Josa',
 '에서/Josa',
 '만/Josa',
 '만들다/Verb',
 '연기/Noun',
 '나오다/Verb',
 '평점/Noun',
 '과/Josa',
 '최고/Noun',
 '~/Punctuation',
 '한/Josa',
 '이다/Verb',
 '왜/Noun',
 '스토리/Noun',
 '생각/Noun',
 '..../Punctuation',
 '인/Josa',
 '드라마/Noun',
 '다/Adverb',
 '다/Josa',
 '이다/Josa',
 '감동/Noun',
 '사람/Noun',
 '1/Number',
 'ㅋㅋㅋ/KoreanParticle',
 '보고/Noun',
 '말/Noun',
 '이렇다/Adjective',
 '아깝다/Adjective',
 '더/Noun',
 '때/Noun',
 'ㅋㅋ/KoreanParticle',
 '내/Noun',
 '배우/Noun',
 '거/Noun',
 '재미있다/Adjective',
 '와/Josa',
 '감독/Nou

In [12]:
def term_frequency(doc):
    return [doc.count(word) for word in selected_words] 

train_x = [term_frequency(d) for d, _ in train_docs]
test_x = [term_frequency(d) for d, _ in test_docs]
train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')
y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')


In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras import losses
from keras import metrics

Using TensorFlow backend.


In [14]:
model = Sequential()
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001), loss=losses.binary_crossentropy, metrics=[metrics.binary_accuracy])

model.fit(x_train, y_train, epochs=10, batch_size=512)
results = model.evaluate(x_test, y_test)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
150000/150000 [==============================] - 5s 36us/step - loss: 0.4320 - binary_accuracy: 0.8018
Epoch 2/10
150000/150000 [==============================] - 5s 34us/step - loss: 0.3811 - binary_accuracy: 0.8241: 1s - loss: 0.3838 - binary_accuracy: 0.822 - ETA: 1s - loss: 0.3836 - bi - ETA: 0s - loss: 0.3810 - binary_accuracy: 0
Epoch 3/10
150000/150000 [==============================] - 4s 27us/step - loss: 0.3651 - binary_accuracy: 0.8332
Epoch 4/10
150000/150000 [==============================] - 4s 24us/step - loss: 0.3519 - binary_accuracy: 0.8411:
Epoch 5/10
150000/150000 [==============================] - 3s 23us/step - loss: 0.3394 - binary_accuracy: 0.8487
Epoch 6/10
150000/150000 [==============================] - 3s 22us/step - loss: 0.3276 - binary_accuracy: 0.8548
Epoch 7/10
150000/150000 [==============================] - 3s 21us/step - 

In [15]:
def predict_pos_neg(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(model.predict(data))
    if(score > 0.5):
        print("[{}]는 {:.2f}% 긍정\n".format(review, score * 100))
    else:
        print("[{}]는 {:.2f}% 부정\n".format(review, (1 - score) * 100))


# 테스트
predict_pos_neg("할줄아는거 없고 해낼 자신도 없는데 한심하고 착잡하다")
predict_pos_neg("슬픈 영화 보고 그냥 펑펑 울고 싶은데 추천 좀 해주세요")
predict_pos_neg("겨드랑이에서 삼겹살 냄새 나는듯")
predict_pos_neg("진짜 너무 심심한데 놀아줄사람")
predict_pos_neg("튜터링 돈 언제 주냐 ㅋㅋㅋㅋㅋ")
predict_pos_neg("노트북 브랜드 추천점")
predict_pos_neg("문제다 문제 우리나라 정치나 학교 총학이나 나 자신이나 전부 문제야....")

[할줄아는거 없고 해낼 자신도 없는데 한심하고 착잡하다]는 95.76% 부정

[슬픈 영화 보고 그냥 펑펑 울고 싶은데 추천 좀 해주세요]는 97.36% 긍정

[겨드랑이에서 삼겹살 냄새 나는듯]는 56.33% 긍정

[진짜 너무 심심한데 놀아줄사람]는 78.00% 부정

[튜터링 돈 언제 주냐 ㅋㅋㅋㅋㅋ]는 89.45% 부정

[노트북 브랜드 추천점]는 59.35% 긍정

[문제다 문제 우리나라 정치나 학교 총학이나 나 자신이나 전부 문제야....]는 88.27% 부정

